In [1]:
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [2]:
filepath = '../Resources/Age_Datasets/gen_X_stroke.csv'
medical_df = pd.read_csv(filepath)
medical_df.head(10)

,ID#,Gender,Age,Hypertension,Heart_Disease,Ever_Married,Work_Type,Residence_Type,Avg_Glucose_Lvl,BMI,Smoker,Stroke
0,4,Female,49,0,0,Yes,Private,Urban,171.23,34.4,Current,1
1,9,Female,59,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
2,13,Female,54,0,0,Yes,Private,Urban,104.51,27.3,Current,1
3,16,Female,50,1,0,Yes,Self-employed,Rural,167.41,30.9,Never,1
4,19,Female,60,0,0,No,Private,Urban,89.22,37.8,Never,1
5,20,Male,57,0,1,No,Govt_job,Urban,217.08,NaN,Unknown,1
6,22,Female,52,1,0,Yes,Self-employed,Urban,233.29,48.9,Never,1
7,28,Male,58,0,0,Yes,Private,Rural,189.84,NaN,Unknown,1
8,30,Male,59,0,0,Yes,Private,Rural,211.78,NaN,Former,1
9,31,Male,57,1,0,Yes,Private,Urban,212.08,44.2,Current,1


In [3]:
# Drop ID# column
medical_df.drop(columns=['ID#'], inplace=True)
medical_df.head()

,Gender,Age,Hypertension,Heart_Disease,Ever_Married,Work_Type,Residence_Type,Avg_Glucose_Lvl,BMI,Smoker,Stroke
0,Female,49,0,0,Yes,Private,Urban,171.23,34.4,Current,1
1,Female,59,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
2,Female,54,0,0,Yes,Private,Urban,104.51,27.3,Current,1
3,Female,50,1,0,Yes,Self-employed,Rural,167.41,30.9,Never,1
4,Female,60,0,0,No,Private,Urban,89.22,37.8,Never,1


In [28]:
medical_df['Stroke'].value_counts()

0    1498
1      64
Name: Stroke, dtype: int64

In [4]:
# Get a list of categorical columns
categorical_columns = medical_df.dtypes[medical_df.dtypes=='object'].index.tolist()
categorical_columns

['Gender', 'Ever_Married', 'Work_Type', 'Residence_Type', 'Smoker']

In [5]:
# Create OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit & transform OneHotEncoder using categorical columns
encode_df = pd.DataFrame(enc.fit_transform(medical_df[categorical_columns]))

# Add column names
encode_df.columns = enc.get_feature_names(categorical_columns)
print(encode_df.shape)
encode_df.head(10)

(1562, 13)


C:\Users\remem\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Private,Work_Type_Self-employed,Residence_Type_Rural,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
6,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
7,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
8,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
9,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [6]:
# Get encode_df info
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562 entries, 0 to 1561
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender_Female            1562 non-null   float64
 1   Gender_Male              1562 non-null   float64
 2   Ever_Married_No          1562 non-null   float64
 3   Ever_Married_Yes         1562 non-null   float64
 4   Work_Type_Govt_job       1562 non-null   float64
 5   Work_Type_Private        1562 non-null   float64
 6   Work_Type_Self-employed  1562 non-null   float64
 7   Residence_Type_Rural     1562 non-null   float64
 8   Residence_Type_Urban     1562 non-null   float64
 9   Smoker_Current           1562 non-null   float64
 10  Smoker_Former            1562 non-null   float64
 11  Smoker_Never             1562 non-null   float64
 12  Smoker_Unknown           1562 non-null   float64
dtypes: float64(13)
memory usage: 158.8 KB


In [7]:
# Drop redundant columns
encode_df.drop(columns=['Gender_Female', 'Ever_Married_No', 'Residence_Type_Rural'], inplace=True)
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562 entries, 0 to 1561
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender_Male              1562 non-null   float64
 1   Ever_Married_Yes         1562 non-null   float64
 2   Work_Type_Govt_job       1562 non-null   float64
 3   Work_Type_Private        1562 non-null   float64
 4   Work_Type_Self-employed  1562 non-null   float64
 5   Residence_Type_Urban     1562 non-null   float64
 6   Smoker_Current           1562 non-null   float64
 7   Smoker_Former            1562 non-null   float64
 8   Smoker_Never             1562 non-null   float64
 9   Smoker_Unknown           1562 non-null   float64
dtypes: float64(10)
memory usage: 122.2 KB


In [8]:
# Merge encoded df with medical df
medical_df = medical_df.merge(encode_df, left_index=True, right_index=True).drop(categorical_columns, axis=1)
print(medical_df.shape)
medical_df.head(10)

(1562, 16)


,Age,Hypertension,Heart_Disease,Avg_Glucose_Lvl,BMI,Stroke,Gender_Male,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Private,Work_Type_Self-employed,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,49,0,0,171.23,34.4,1,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
1,59,0,0,76.15,NaN,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,54,0,0,104.51,27.3,1,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
3,50,1,0,167.41,30.9,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,60,0,0,89.22,37.8,1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
5,57,0,1,217.08,NaN,1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
6,52,1,0,233.29,48.9,1,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
7,58,0,0,189.84,NaN,1,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
8,59,0,0,211.78,NaN,1,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9,57,1,0,212.08,44.2,1,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


In [9]:
# Create SimpleImputer instance to replace missing BMI feature values with median BMI
imputer = SimpleImputer(strategy='mean')

# Fit SimpleImputer & transform data
med_transformed = imputer.fit_transform(medical_df)

In [10]:
# Add SimpleImputer outcome to dataframe
med_df_transformed = pd.DataFrame(med_transformed, columns=medical_df.columns)
print(med_df_transformed.shape)
med_df_transformed.head(10)

(1562, 16)


,Age,Hypertension,Heart_Disease,Avg_Glucose_Lvl,BMI,Stroke,Gender_Male,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Private,Work_Type_Self-employed,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,49.0,0.0,0.0,171.23,34.400000,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
1,59.0,0.0,0.0,76.15,31.668435,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,54.0,0.0,0.0,104.51,27.300000,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
3,50.0,1.0,0.0,167.41,30.900000,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,60.0,0.0,0.0,89.22,37.800000,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
5,57.0,0.0,1.0,217.08,31.668435,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
6,52.0,1.0,0.0,233.29,48.900000,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
7,58.0,0.0,0.0,189.84,31.668435,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
8,59.0,0.0,0.0,211.78,31.668435,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9,57.0,1.0,0.0,212.08,44.200000,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


In [11]:
med_df_transformed.isnull().sum()

Age                        0
Hypertension               0
Heart_Disease              0
Avg_Glucose_Lvl            0
BMI                        0
Stroke                     0
Gender_Male                0
Ever_Married_Yes           0
Work_Type_Govt_job         0
Work_Type_Private          0
Work_Type_Self-employed    0
Residence_Type_Urban       0
Smoker_Current             0
Smoker_Former              0
Smoker_Never               0
Smoker_Unknown             0
dtype: int64

In [12]:
# Create feature & target datasets
X = med_df_transformed.drop(columns=['Stroke'])
y = med_df_transformed['Stroke']
print(X.shape)
print(y.shape)

(1562, 15)
(1562,)


In [13]:
# Create empty arrays to store ML results
recalls = []
accuracies = []
precisions = []

# This will take a while. Feel free to adjust range to sample code. Feel free to review exported spreadsheets in github.
for i in range(1000):
    
    # Create training + testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    
    # Oversample positive stroke cases in training set
    ros = RandomOverSampler()
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    
    # Scale feature data
    scaler = StandardScaler()
    scaler.fit(X_train_resampled)
    X_train_scaled = scaler.transform(X_train_resampled)
    X_test_scaled = scaler.transform(X_test)
    
    # Create SVC model
    svc_model = SVC(kernel='linear')
    svc_model = svc_model.fit(X_train_scaled, y_train_resampled)
    
    # Evaluate SVC model
    y_pred = svc_model.predict(X_test_scaled)
    
    # Add recall results to list
    recalls.append(recall_score(y_test, y_pred))
    
    # Add precision results to list
    precisions.append(precision_score(y_test, y_pred))
    
    # Add accuracy results to list
    accuracies.append(accuracy_score(y_test, y_pred))
    
    print("Current progress:", i)

Current progress: 0
Current progress: 1
Current progress: 2
Current progress: 3
Current progress: 4
Current progress: 5
Current progress: 6
Current progress: 7
Current progress: 8
Current progress: 9
Current progress: 10
Current progress: 11
Current progress: 12
Current progress: 13
Current progress: 14
Current progress: 15
Current progress: 16
Current progress: 17
Current progress: 18
Current progress: 19
Current progress: 20
Current progress: 21
Current progress: 22
Current progress: 23
Current progress: 24
Current progress: 25
Current progress: 26
Current progress: 27
Current progress: 28
Current progress: 29
Current progress: 30
Current progress: 31
Current progress: 32
Current progress: 33
Current progress: 34
Current progress: 35
Current progress: 36
Current progress: 37
Current progress: 38
Current progress: 39
Current progress: 40
Current progress: 41
Current progress: 42
Current progress: 43
Current progress: 44
Current progress: 45
Current progress: 46
Current progress: 47
Cu

Current progress: 378
Current progress: 379
Current progress: 380
Current progress: 381
Current progress: 382
Current progress: 383
Current progress: 384
Current progress: 385
Current progress: 386
Current progress: 387
Current progress: 388
Current progress: 389
Current progress: 390
Current progress: 391
Current progress: 392
Current progress: 393
Current progress: 394
Current progress: 395
Current progress: 396
Current progress: 397
Current progress: 398
Current progress: 399
Current progress: 400
Current progress: 401
Current progress: 402
Current progress: 403
Current progress: 404
Current progress: 405
Current progress: 406
Current progress: 407
Current progress: 408
Current progress: 409
Current progress: 410
Current progress: 411
Current progress: 412
Current progress: 413
Current progress: 414
Current progress: 415
Current progress: 416
Current progress: 417
Current progress: 418
Current progress: 419
Current progress: 420
Current progress: 421
Current progress: 422
Current pr

Current progress: 751
Current progress: 752
Current progress: 753
Current progress: 754
Current progress: 755
Current progress: 756
Current progress: 757
Current progress: 758
Current progress: 759
Current progress: 760
Current progress: 761
Current progress: 762
Current progress: 763
Current progress: 764
Current progress: 765
Current progress: 766
Current progress: 767
Current progress: 768
Current progress: 769
Current progress: 770
Current progress: 771
Current progress: 772
Current progress: 773
Current progress: 774
Current progress: 775
Current progress: 776
Current progress: 777
Current progress: 778
Current progress: 779
Current progress: 780
Current progress: 781
Current progress: 782
Current progress: 783
Current progress: 784
Current progress: 785
Current progress: 786
Current progress: 787
Current progress: 788
Current progress: 789
Current progress: 790
Current progress: 791
Current progress: 792
Current progress: 793
Current progress: 794
Current progress: 795
Current pr

In [14]:
recalls

[0.5384615384615384,
 0.6153846153846154,
 0.5384615384615384,
 0.5384615384615384,
 0.5384615384615384,
 0.7692307692307693,
 0.46153846153846156,
 0.6153846153846154,
 0.46153846153846156,
 0.6923076923076923,
 0.8461538461538461,
 0.5384615384615384,
 0.46153846153846156,
 0.8461538461538461,
 0.6923076923076923,
 0.8461538461538461,
 0.6153846153846154,
 0.9230769230769231,
 0.6153846153846154,
 0.46153846153846156,
 0.8461538461538461,
 0.7692307692307693,
 0.6153846153846154,
 0.46153846153846156,
 0.6923076923076923,
 0.6153846153846154,
 0.38461538461538464,
 0.6153846153846154,
 0.5384615384615384,
 0.6153846153846154,
 0.7692307692307693,
 0.6153846153846154,
 0.6923076923076923,
 0.7692307692307693,
 0.8461538461538461,
 0.46153846153846156,
 0.8461538461538461,
 0.5384615384615384,
 0.6153846153846154,
 0.6923076923076923,
 0.38461538461538464,
 0.6153846153846154,
 0.5384615384615384,
 0.6153846153846154,
 0.38461538461538464,
 0.46153846153846156,
 0.6923076923076923,
 0.

In [15]:
recalls_df = pd.DataFrame(recalls, columns=['Recall'])
print(recalls_df.shape)
recalls_df.head(10)

(1000, 1)


,Recall
0,0.538462
1,0.615385
2,0.538462
3,0.538462
4,0.538462
5,0.769231
6,0.461538
7,0.615385
8,0.461538
9,0.692308


In [16]:
accuracies

[0.6357827476038339,
 0.6421725239616614,
 0.597444089456869,
 0.6325878594249201,
 0.5910543130990416,
 0.6485623003194888,
 0.5559105431309904,
 0.5942492012779552,
 0.610223642172524,
 0.6134185303514377,
 0.6773162939297125,
 0.6357827476038339,
 0.6134185303514377,
 0.6006389776357828,
 0.6038338658146964,
 0.6261980830670927,
 0.5814696485623003,
 0.6293929712460063,
 0.6006389776357828,
 0.597444089456869,
 0.6357827476038339,
 0.5591054313099042,
 0.5878594249201278,
 0.6293929712460063,
 0.65814696485623,
 0.610223642172524,
 0.6389776357827476,
 0.6166134185303515,
 0.6645367412140575,
 0.6198083067092651,
 0.5846645367412141,
 0.5750798722044729,
 0.6166134185303515,
 0.6389776357827476,
 0.6645367412140575,
 0.65814696485623,
 0.5782747603833865,
 0.6741214057507987,
 0.6517571884984026,
 0.6166134185303515,
 0.6389776357827476,
 0.6134185303514377,
 0.5654952076677316,
 0.5942492012779552,
 0.6357827476038339,
 0.670926517571885,
 0.6261980830670927,
 0.6293929712460063,
 

In [17]:
accuracies_df = pd.DataFrame(accuracies, columns=['Accuracy'])
print(accuracies_df.shape)
accuracies_df.head(10)

(1000, 1)


,Accuracy
0,0.635783
1,0.642173
2,0.597444
3,0.632588
4,0.591054
5,0.648562
6,0.555911
7,0.594249
8,0.610224
9,0.613419


In [18]:
precisions

[0.06086956521739131,
 0.06956521739130435,
 0.05511811023622047,
 0.0603448275862069,
 0.05426356589147287,
 0.08547008547008547,
 0.043478260869565216,
 0.06153846153846154,
 0.049586776859504134,
 0.07142857142857142,
 0.1,
 0.06086956521739131,
 0.05,
 0.08208955223880597,
 0.06976744186046512,
 0.0873015873015873,
 0.05970149253731343,
 0.09448818897637795,
 0.0625,
 0.048,
 0.08943089430894309,
 0.06896551724137931,
 0.06060606060606061,
 0.05217391304347826,
 0.08035714285714286,
 0.064,
 0.045454545454545456,
 0.06504065040650407,
 0.0660377358490566,
 0.06557377049180328,
 0.072992700729927,
 0.058823529411764705,
 0.072,
 0.08333333333333333,
 0.09649122807017543,
 0.05660377358490566,
 0.07801418439716312,
 0.06796116504854369,
 0.07142857142857142,
 0.072,
 0.045454545454545456,
 0.06451612903225806,
 0.051094890510948905,
 0.06153846153846154,
 0.04504504504504504,
 0.058823529411764705,
 0.07377049180327869,
 0.088,
 0.05925925925925926,
 0.05102040816326531,
 0.056074766

In [19]:
precisions_df = pd.DataFrame(precisions, columns=['Precision'])
print(precisions_df.shape)
precisions_df.head(10)

(1000, 1)


,Precision
0,0.060870
1,0.069565
2,0.055118
3,0.060345
4,0.054264
5,0.085470
6,0.043478
7,0.061538
8,0.049587
9,0.071429


In [20]:
results_df = accuracies_df.merge(precisions_df, left_index=True, right_index=True)
print(results_df.shape)
results_df.head(10)

(1000, 2)


,Accuracy,Precision
0,0.635783,0.060870
1,0.642173,0.069565
2,0.597444,0.055118
3,0.632588,0.060345
4,0.591054,0.054264
5,0.648562,0.085470
6,0.555911,0.043478
7,0.594249,0.061538
8,0.610224,0.049587
9,0.613419,0.071429


In [21]:
results_df = results_df.merge(recalls_df, left_index=True, right_index=True)
print(results_df.shape)
results_df.head(10)

(1000, 3)


,Accuracy,Precision,Recall
0,0.635783,0.060870,0.538462
1,0.642173,0.069565,0.615385
2,0.597444,0.055118,0.538462
3,0.632588,0.060345,0.538462
4,0.591054,0.054264,0.538462
5,0.648562,0.085470,0.769231
6,0.555911,0.043478,0.461538
7,0.594249,0.061538,0.615385
8,0.610224,0.049587,0.461538
9,0.613419,0.071429,0.692308


In [22]:
print("Recall mean: " + str(results_df['Recall'].mean()))
print("Recall mode: " + str(results_df['Recall'].mode()))
print("Recall median: " + str(results_df['Recall'].median()))
print("Recall max: " + str(results_df['Recall'].max()))
print("Recall min: " + str(results_df['Recall'].min()))
print("Recall standard deviation: " + str(results_df['Recall'].std()))

Recall mean: 0.604000000000001
Recall mode: 0    0.615385
dtype: float64
Recall median: 0.6153846153846154
Recall max: 1.0
Recall min: 0.15384615384615385
Recall standard deviation: 0.13445377444472456


In [23]:
print("Accuracy mean: " + str(results_df['Accuracy'].mean()))
print("Accuracy mode: " + str(results_df['Accuracy'].mode()))
print("Accuracy median: " + str(results_df['Accuracy'].median()))
print("Accuracy max: " + str(results_df['Accuracy'].max()))
print("Accuracy min: " + str(results_df['Accuracy'].min()))
print("Accuracy standard deviation: " + str(results_df['Accuracy'].std()))

Accuracy mean: 0.6255239616613426
Accuracy mode: 0    0.619808
dtype: float64
Accuracy median: 0.6261980830670927
Accuracy max: 0.7060702875399361
Accuracy min: 0.5335463258785943
Accuracy standard deviation: 0.03019089485848201


In [24]:
print("Precision mean: " + str(results_df['Precision'].mean()))
print("Precision mode: " + str(results_df['Precision'].mode()))
print("Precision median: " + str(results_df['Precision'].median()))
print("Precision max: " + str(results_df['Precision'].max()))
print("Precision min: " + str(results_df['Precision'].min()))
print("Precision standard deviation: " + str(results_df['Precision'].std()))

Precision mean: 0.06531448733043553
Precision mode: 0    0.071429
dtype: float64
Precision median: 0.06542056074766354
Precision max: 0.10344827586206896
Precision min: 0.018018018018018018
Precision standard deviation: 0.012712833597697525


In [25]:
results_df.isnull().sum().sum()

0

In [26]:
len(results_df)

1000

In [27]:
filepath = ('ML_result_files/SVC_GenX_SimpleImputer(mean)_RandomOverSampler.csv')
results_df.to_csv(filepath, index=False)